In [39]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
import mlflow
import vecnn
import time

def mlflow_all_experiments(tracking_uri="./mlruns"):
    client = mlflow.tracking.MlflowClient(tracking_uri=tracking_uri)
    experiment_ids = [e.experiment_id for e in client.search_experiments()]
    records = []
    for id in experiment_ids:
        records.extend(mlflow_experiment_records(id, client))
    df = pd.DataFrame.from_records(records)
    return df

def mlflow_experiment(name: str, tracking_uri="./mlruns") -> pd.DataFrame | None:
    client = mlflow.tracking.MlflowClient(tracking_uri=tracking_uri)
    experiment_ids = [e.experiment_id for e in client.search_experiments() if e.name == name] # filter_string=f"name = '{name}'"
    if len(experiment_ids) == 0:
        return None
    records = mlflow_experiment_records(experiment_ids[0], client)
    df = pd.DataFrame.from_records(records)
    return df

# remove stupid columns
def mlflow_experiment_stripped(name: str, tracking_uri="./mlruns") -> pd.DataFrame | None:
    df = mlflow_experiment(name=name, tracking_uri=tracking_uri)
    if df is None:
        return None
    forbidden_cols = ["ex_artifact_location", "ex_creation_time", "ex_experiment_id", "ex_last_update_time", "ex_lifecycle_stage", "ex_tags", "tag_mlflow.source.type", "tag_mlflow.user", "tag_mlflow.runName", "tag_mlflow.source.name"]
    df = df.drop(columns=[col for col in forbidden_cols if col in df.columns])
    return df


def mlflow_experiment_records(id: str, client: mlflow.client.MlflowClient) -> dict:
    records = []
    exp = client.get_experiment(id)
    runs = client.search_runs(id)
    ex_record = dict()
    for (k,v) in exp:
        ex_record[f"ex_{k}"] = v

    for r in runs:
        record = {**ex_record}
        for k in r.data.params:
            record[f"p_{k}"] = r.data.params[k]
        for k in r.data.metrics:
            record[f"m_{k}"] = r.data.metrics[k]
        for k in r.data.tags:
            record[f"tag_{k}"] = r.data.tags[k]
        records.append(record)
    return records



def mlflow_fresh_experiment(experiment_name: str): 
    mlflow.set_tracking_uri("./mlruns")

    existing = mlflow.search_experiments(filter_string=f"name = '{experiment_name}'")
    if len(existing) != 0:
        mlflow.delete_experiment(existing[0].experiment_id)
    mlflow.set_experiment(experiment_name=experiment_name)


# ndc = number of distance calculations
def mlflow_log_model(model_name: str, data_n: str, data_dims: str, knn_k: str, build_time: float, build_ndc: float | None, search_time: float, search_ndc: float | None, search_recall: float, **kwargs):
    with mlflow.start_run():
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("data_n", data_n)
        mlflow.log_param("data_dims", data_dims)
        mlflow.log_param("knn_k", knn_k)
        if build_ndc is not None:
            mlflow.log_metric("build_ndc", build_ndc)
        mlflow.log_metric("build_time", build_time)
        mlflow.log_metric("search_time", search_time)
        if search_ndc is not None:
            mlflow.log_metric("search_ndc", search_ndc)
        mlflow.log_metric("search_recall", search_recall)
        
        for key, value in kwargs.items():
            mlflow.log_param(key, value)

In [52]:
experiment_name = "explore"
mlflow_fresh_experiment(experiment_name)

ns = [2000]
dims = [780]
n_queries = 100
knn_ks = [20]

for n in ns:
    for dim in dims:
        data_np = np.random.random((n,dim)).astype("float32")
        data = vecnn.Dataset(data_np)
        queries = np.random.random((n_queries,dim)).astype("float32")
        
        start = time.time()
        vp_tree = vecnn.VpTree(data)
        vp_tree_build_time =  time.time() - start

        start = time.time()
        hnsw = vecnn.Hnsw(data, params=vecnn.HnswParams(0.7, 20, 8, 8))
        hnsw_build_time =  time.time() - start

        start = time.time()
        scipy_kd_tree = cKDTree(data_np)
        scipy_kd_tree_build_time =  time.time() - start
        
        for k in knn_ks:
            truth_indices = np.zeros((n_queries,k)).astype("uint64")
            
            # linear search run: 
            search_time = 0
            for i in range(n_queries):
                start = time.time()
                res = vecnn.linear_knn(data, queries[i,:], k)
                search_time += time.time() - start
                truth_indices[i:] = res.indices
            search_time /= n_queries
            mlflow_log_model(model_name = "linear", 
                             data_n=n, 
                             data_dims = dim,
                             knn_k = k,
                             build_time = 0,
                             build_ndc=0,
                             search_time=search_time,
                             search_ndc=n,
                             search_recall=1.0)
        
            # vptree: 
            search_time = 0
            recall = 0
            ndc = 0
            for i in range(n_queries):
                start = time.time()
                res = vp_tree.knn(queries[i,:], k)
                search_time += time.time() - start
                recall += vecnn.knn_recall(truth_indices[i,:], res.indices)
                ndc += res.num_distance_calculations
            search_time /= n_queries
            recall /= n_queries
            ndc /= n_queries
            mlflow_log_model(model_name = "vp_tree", 
                             data_n = n, 
                             data_dims = dim,
                             knn_k = k,
                             build_time = vp_tree_build_time,
                             build_ndc = vp_tree.num_distance_calculations_in_build,
                             search_time=search_time,
                             search_ndc=ndc,
                             search_recall=recall)

            # hnsw:
            search_time = 0
            recall = 0
            ndc = 0
            for i in range(n_queries):
                start = time.time()
                res = hnsw.knn(queries[i,:], k)
                search_time += time.time() - start
                recall += vecnn.knn_recall(truth_indices[i,:], res.indices)
                ndc += res.num_distance_calculations
            search_time /= n_queries
            recall /= n_queries
            ndc /= n_queries
            mlflow_log_model(model_name = "hnsw", 
                             data_n = n, 
                             data_dims = dim,
                             knn_k = k,
                             build_time = hnsw_build_time,
                             build_ndc = hnsw.num_distance_calculations_in_build,
                             search_time=search_time,
                             search_ndc=ndc,
                             search_recall=recall)
            
            # scipy kdtree:
            search_time = 0
            recall = 0
            for i in range(n_queries):
                start = time.time()
                _, indices = scipy_kd_tree.query(queries[i,:], k=k, workers=8)
                search_time += time.time() - start
                recall += 1.0 # vecnn.knn_recall(truth_indices[i,:], indices)
            search_time /= n_queries
            recall /= n_queries
            mlflow_log_model(model_name = "scipy_kd_tree", 
                             data_n = n, 
                             data_dims = dim,
                             knn_k = k,
                             build_time = scipy_kd_tree_build_time,
                             build_ndc = None,
                             search_time=search_time,
                             search_ndc = None,
                             search_recall=recall)

df = mlflow_experiment_stripped("explore")
df

2024/05/19 15:35:27 INFO mlflow.tracking.fluent: Experiment with name 'explore' does not exist. Creating a new experiment.


,ex_name,p_data_n,p_model_name,p_knn_k,p_data_dims,m_search_recall,m_build_time,m_search_time,m_build_ndc,m_search_ndc
0,explore,2000,scipy_kd_tree,20,780,1.000,0.008689,0.001094,NaN,NaN
1,explore,2000,hnsw,20,780,0.463,3.050063,0.001738,714328.0,411.99
2,explore,2000,vp_tree,20,780,1.000,0.064111,0.007691,18290.0,2000.00
3,explore,2000,linear,20,780,1.000,0.000000,0.007276,0.0,2000.00


,ex_name,p_data_n,p_model_name,p_knn_k,p_data_dims,m_search_recall,m_build_time,m_search_time,m_build_ndc,m_search_ndc
0,explore,1000,vp_tree,100,3,0.83726,0.001526,0.000071,8150.0,217.141
1,explore,1000,linear,100,3,1.00000,0.000000,0.000230,0.0,1000.000
2,explore,1000,vp_tree,10,3,0.74580,0.001526,0.000013,8150.0,36.971
3,explore,1000,linear,10,3,1.00000,0.000000,0.000165,0.0,1000.000
4,explore,1000,vp_tree,1,3,0.63200,0.001526,0.000005,8150.0,13.148
5,explore,1000,linear,1,3,1.00000,0.000000,0.000153,0.0,1000.000
6,explore,1000,vp_tree,100,2,0.88374,0.001569,0.000058,8150.0,181.876
7,explore,1000,linear,100,2,1.00000,0.000000,0.000220,0.0,1000.000
8,explore,1000,vp_tree,10,2,0.80260,0.001569,0.000011,8150.0,28.534
9,explore,1000,linear,10,2,1.00000,0.000000,0.000161,0.0,1000.000


In [ ]:

n = 1000
dims = 600
data = np.random .random((n,dims)).astype("float32")


ds = Dataset(data)

start = time.time()
tree = VpTree(ds)
# print(tree.num_distance_calculations_in_build)
print("vp-tree build:", time.time()-start)

start = time.time()
hnsw = Hnsw(ds, HnswParams(0.6, 6, 3, 4))
# print(hnsw.num_distance_calculations_in_build)
print("hnsw build:", time.time()-start)

start = time.time()
pytree = cKDTree(data)
print("cKDTree build:", time.time()-start)

q = np.random.random((dims,)).astype("float32")
k = 100

start = time.time()
for i in range(1000):
    res = linear_knn(ds, q, k)
print("linear query:", time.time()-start)
print("linear", (res.indices, res.distances, res.num_distance_calculations))

start = time.time()
for i in range(1000):
    res =tree.knn(q, k)
print("vp-tree query:", time.time()-start)
print((res.indices, res.distances, res.num_distance_calculations) )